# Getting started with Starmon-7

**Authors:** Marios Samiotis (m.samiotis@tudelft.nl)

**Date:** February 14, 2025

# 1. Introduction

In this notebook we will get started with using the Starmon-7 superconducting backend which exists at the [Quantum Inspire 2.0](https://www.quantum-inspire.com/) cloud services!

For optimal performance, make sure that you are running this notebook within a customized Python 3.12 environment which includes the packages "quantuminspire" and "qiskit-quantuminspire".

For detailed instructions on how to create such a Python environment, follow the instructions in the [README file](https://github.com/DiCarloLab-Delft/QuantumInspireUtilities/blob/main/README.md).

Useful links:
1. [Starmon-7 Fact Sheet](https://github.com/DiCarloLab-Delft/QuantumInspireUtilities/blob/main/Starmon7_FactSheet.pdf)
2. [Starmon-7 Performance Dashboard](https://monitoring.qutech.support/public-dashboards/7171f0e3cfc44995a97dff9001c4d7d1?orgId=1) [live]
3. [Starmon-7 Performance Metrics](https://dicarlolab.tudelft.nl/Starmon7_performance.html) [updated once a day]

First, we will run the following cell in order to login to the Quantum Inspire platform. You will need an account in order to login to the platform.

Please click on "Confirm" in the pop-up browser window.

In [ ]:
! qi login "https://api.quantum-inspire.com"

Run the following cell for all necessary library imports,

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from qiskit import QuantumCircuit, transpile
from qiskit_quantuminspire.qi_provider import QIProvider
from qi_utilities.utility_functions.circuit_modifiers import apply_readout_circuit
from qi_utilities.utility_functions.ro_correction import (split_raw_shots, extract_ro_assignment_matrix, plot_ro_assignment_matrix,
                                                          ro_corrected_multi_qubit_prob)
from qi_utilities.utility_functions.midcircuit_msmt import get_multi_qubit_counts, get_multi_qubit_prob

# 2. Connect to the Quantum Inspire backend Starmon-7

We will first need to connect to the Quantum Inspire provider, by running the following cell,

In [ ]:
provider = QIProvider()

You may see the full list of all available backends of Quantum Inspire platform by running

In [ ]:
provider.backends()

We now connect to the backend Starmon-7 backend by running the following cell,

In [ ]:
backend_name = "Starmon-7"
backend = provider.get_backend(name=backend_name)

Starmon-7 is a superconducting quantum processor based on circuit quantum electrodynamics. It consists of seven transmon qubits, labelled Q0 to Q6, with nearest-neighbor connectivity intended for the [distance-2 surface code](https://doi.org/10.1038/s41567-021-01423-9).

By running the cell below, one can print the connectivity (otherwise known as "coupling map") of the processor,

In [ ]:
backend.coupling_map.draw()

We refer to any single- or multi-qubit gates that one may apply on the qubits of Starmon-7 as "instructions". One may get the full list of allowed instructions on Starmon-7 backend by running the following cell,

In [ ]:
backend.instructions

When running a quantum algorithm one needs to choose a number of shots to be performed for the given quantum circuit. The maximum number of allowed shots on Starmon-7 can be obtained by running the command

In [ ]:
backend.max_shots

# 3. Running your first quantum circuit on Starmon-7

Here we will write a quantum circuit which creates a Bell-pair state between two qubits. For more information concerning what a Bell-pair state is, visit [https://www.quantum-inspire.com/kbase/hello-quantum-world/](https://www.quantum-inspire.com/kbase/hello-quantum-world/).

To achieve this, we have to apply a two-qubit gate between two qubits that are nearest neighbors. We can write such a program in two ways: either we take into account the number of qubits and connectivity of the device while writing the code, or we run a very simple program consisting of two qubits and we let the Qiskit transpiler take care of mapping of our operations on Starmon-7.

In Section 3.1. we will create a quantum circuit with a quantum register of 7 qubits, respecting the connectivity of the device, while in Section 3.2 we will use the Qiskit transpiler.

## 3.1. Creating a Bell-pair state without the use of Qiskit transpiler

Since we are not using the Qiskit transpiler, we have to be careful and define the correct number of qubits, as well as perform two-qubit gates on qubits that are connected together (see the coupling map of the processor above).

In addition, one may consult the Starmon-7 performance webpage, [https://dicarlolab.tudelft.nl/Starmon7_performance.html](https://dicarlolab.tudelft.nl/Starmon7_performance.html), in order to be informed for which qubit pair to choose: the one with the best reported two-qubit fidelity. Be careful not to use any qubit-pair that may have been marked with red color, indicating that it is insufficiently tuned up due to calibration errors.

At the time of writing this guide, the qubit-pair Q3-Q6 had the best reported performance metrics, and was therefore being chosen,

In [ ]:
qubit_0 = 0
qubit_1 = 2

qc = QuantumCircuit(7, 2)

qc.h(qubit_0)
qc.cx(qubit_0, qubit_1)
qc.measure(qubit_0, cbit=0)
qc.measure(qubit_1, cbit=1)

You may visualize the above quantum circuit by running the following cell,

In [ ]:
qc.draw('mpl')

Now we will run the above circuit on the Starmon-7 backend. We choose for the number of circuit shots the maximum allowed, which as we have already seen in Section 2. is given by "backend.max_shots".

Usually for regular-sized jobs and a normal queue, the Bell-state algorithm should take a few seconds of run time, and the execution result should be retrieved within a minute. In any case, it is a safe practice to request a timeout of about 10 minutes for an unusually busy queue.

In [ ]:
nr_shots = backend.max_shots
job = backend.run(qc, shots=nr_shots)
result = job.result(timeout = 600) # timeout is in seconds

We now gather the measurement counts and visualize the results,

In [ ]:
counts = result.get_counts()

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4), dpi=300)

for bit_string in counts:
    ax.bar(bit_string, counts[bit_string] / nr_shots, color='orange')

ax.set_xlabel("Bit-strings")
ax.set_ylabel("Probabilities")
ax.set_title(f"Bell-state preparation\nQubits Q{qubit_0} - Q{qubit_1}")

ax.set_yticks([0.0, 0.25, 0.5, 0.75, 1.0])
ax.set_ylim(0.0, 1.05)

plt.grid(axis='y')
plt.show()

## 3.2. Creating a Bell-pair state by using the Qiskit transpiler and applying readout error mitigation (ADVANCED)

Here we will be using Qiskit's "transpile" function, which simplifies circuit writing since we do not need to bother with defining the number of qubits of the backend, or take into consideration the coupling map and allowed set of instructions of Starmon-7; the transpiler takes care of all that for us. Additionally, we will be post processing the raw data in order to mitigate readout errors that occur during state measurement.

By taking a look at the previous example, we see that the quantum circuit uses a CNOT gate between two qubits. Though, looking at the Starmon-7 [fact sheet](https://github.com/DiCarloLab-Delft/QuantumInspireUtilities/blob/main/Starmon7_FactSheet.pdf), we read in section VI "Qubit operations" that the only native two-qubit gate in Starmon-7 is the CZ gate. What does that mean in practice?

Whenever we send a quantum circuit to Starmon-7 which includes a CNOT, the CNOT is further decomposed by the Starmon-7 internal software into a combination of single-qubit gates and a CZ gate, a transpilation step which is not visible to the user.

If the user is concerned with optimizing their circuits so that they send jobs to Starmon-7 with circuits which do not need to be further decomposed by the Starmon-7 internal software, then they need to define the following basis gates list which corresponds to the accurate set of allowed instructions of the device, which we will then pass to the Qiskit transpiler manually,

In [ ]:
starmon7_basis_gates = ['id', 'z', 's', 'sdg', 't', 'tdg', 'x', 'rx', 'y', 'ry', 'cz']

Now let us define once again the Bell pair state preparation quantum circuit. Using the mid-circuit measurement functionality, we append the necessary measurements, used later on for mitigating readout errors, at the end of the original circuit,

In [ ]:
qubit_0 = 0
qubit_1 = 1

qc = QuantumCircuit(2, 2)

qc.h(qubit_0)
qc.cx(qubit_0, qubit_1)
qc.measure(qubit_0, cbit=0)
qc.measure(qubit_1, cbit=1)

qc = apply_readout_circuit(qc, [qubit_0, qubit_1])

In [ ]:
qc.draw('mpl')

In order to run the above circuit on Starmon-7, we will need to transpile it.

But how does the transpiler know which qubit pair to choose? If we do not specify it, it will begin with the first entry of the coupling map of the device, which is qubit pair Q0-Q2, and so on.

Nevertheless, we have the freedom to define a qubit priority list: by consulting the performance page [https://dicarlolab.tudelft.nl/Starmon7_performance.html](https://dicarlolab.tudelft.nl/Starmon7_performance.html), we can create a list of qubits with the best performance, followed by the least favorable qubits.

At the day of writing this guide, the best list seemed to be qubit_priority_list = [3, 6, 4, 1, 5, 0, 2]

In [ ]:
qubit_priority_list = [0, 2, 4, 1, 5, 3, 6]

In [ ]:
qc_transpiled = transpile(qc, backend, initial_layout=qubit_priority_list[0:qc.num_qubits], basis_gates=starmon7_basis_gates)

In [ ]:
qc_transpiled.draw('mpl')

We can see that the transpiler added an additional 5 qubits itself, which are referenced as "ancilla" qubits, since they are not being used in the algorithm.

In the above circuit, we see the further decomposition of the CNOT gate into single-qubit rotations and a CZ gate. We can be sure now that the above circuit is the actual instruction list that Starmon-7 will eventually execute.

We will now run the transpiled circuit on Starmon-7,

In [ ]:
nr_shots = backend.max_shots
job = backend.run(qc_transpiled, shots=nr_shots, memory = True)
result = job.result(timeout = 600)

In [ ]:
qubit_list = [qubit_0, qubit_1]

raw_data_shots, ro_mitigation_shots = split_raw_shots(result, qubit_list)
ro_assignment_matrix = extract_ro_assignment_matrix(ro_mitigation_shots, qubit_list)

raw_data_counts = get_multi_qubit_counts(raw_data_shots, len(qubit_list))
raw_data_probs = get_multi_qubit_prob(raw_data_counts)
ro_corrected_probs = ro_corrected_multi_qubit_prob(raw_data_probs, ro_assignment_matrix, qubit_list)

In [ ]:
plot_ro_assignment_matrix(ro_assignment_matrix, [qubit_0, qubit_1])

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4), dpi=300)

bit_strings = [bit_string for bit_string in raw_data_probs[0]]

ax.bar(bit_strings,
       [raw_data_probs[0][bit_string] for bit_string in raw_data_probs[0]],
       color='blue',
       label='Raw data',
       alpha=0.5, align='edge', width=-0.4)
ax.bar(bit_strings,
       [ro_corrected_probs[0][bit_string] for bit_string in ro_corrected_probs[0]],
       color='orange',
       label='Readout-error-mitigation data',
       align='edge', width=0.4)

ax.set_xlabel("Bit-strings")
ax.set_ylabel("Probabilities")
ax.set_title(f"Bell-state preparation\nQubits Q{qubit_priority_list[0]} - Q{qubit_priority_list[1]}")

ax.set_yticks([0.0, 0.25, 0.5, 0.75, 1.0])
ax.set_ylim(0.0, 1.05)
ax.legend()

plt.grid(axis='y')
plt.show()

# 4. Performing a qubit $T_1$ measurement using mid-circuit measurements

Here we will perform a simple measurement in order to estimate the relaxation time $T_1$ of a qubit, using the mid-circuit functionality of the Quantum Inspire SDK. For more information concerning what the qubit relaxation time is or how do we measure it, take a look at [A Quantum Engineer's Guide to Superconducting Qubits](https://arxiv.org/abs/1904.06560v5), Section III. B. 2.

We will first define the experiment parameters,

In [ ]:
qubit_nr = 6 # e.g. the entry "0" is for qubit "Q0"
total_time = 120e-6 # total measurement time in units of [s]
nr_points = 31 # number of measurement points

and then we define the experiment circuit. The user does not need to alter the following code,

In [ ]:
cycle_time = 20e-9 # cycle time of the Central Controller (CC) instrument
dt = total_time / nr_points
measurement_times = np.linspace(start = 0.0, stop = total_time, num = nr_points)

qc = QuantumCircuit(7, nr_points)
qc.x(qubit_nr) # qubit initialization to the |1> state
qc.measure(qubit = qubit_nr, cbit = 0) # initial measurement at time t=0
for time_idx in range(1, nr_points):
    qc.delay(duration = int((dt / cycle_time)), qarg = qubit_nr) # delay in units of CC cycles
    qc.measure(qubit = qubit_nr, cbit = time_idx)

qc = apply_readout_circuit(qc, [qubit_nr])

In [ ]:
qc.draw('mpl')

Let us understand the above circuit. A $T_1$ measurement begins with an initial pi-pulse (X-gate) on the measured qubit, followed by an idle time which we refer to as the "total_time", where we sample the qubit state for a number of "nr_points".

The "total_time" of the experiment is chosen to be approximately 4 times the qubit relaxation time $T_1$, which was already known to us at the time of writing this guide, and thus we chose $150 \mu s$ as the total duration of the experiment.

You will see in the circuit multiple measurement blocks, equal to the "nr_points". These blocks are what we refer to as "mid-circuit measurements", since after each measurement block the circuit continues, until the final circuit measurement.

How do the results of such a measurement look like? We save the outcome of each measurement block to a separate bit in the bit register, which has a total size equal to the "nr_points".

Notice how in the method ".run()" we set "memory = True", which returns the raw data of the experiment.

In [ ]:
nr_shots = backend.max_shots # NOTE: one has to be careful with nr_shots using mid-circuit measurements
                             #       if the job fails to be executed, reduce the nr_shots
job = backend.run(qc, shots=nr_shots, memory = True) # NOTE: memory is set to True in order to return raw data!
result = job.result(timeout = 600)

Let us now investigate how the data is organized by running the custom function "return_raw_data()" from utility_functions.py,

In [ ]:
raw_data_shots, ro_mitigation_shots = split_raw_shots(result, [qubit_nr])

The raw_data list has a total "nr_shots" entries, since we run this circuit for that particular number of shots, and each entry has "nr_points" bits, with each containing the result of each measurement block for the particular circuit run.

The right-most bit of each bit string corresponds to the first measurement block of the quantum circuit, while the left-most bit of each bit string corresponds to the final measurement.

For example, the entry "raw_data[0][-1]" gives us the result of the first measurement block of the very first circuit shot. You will notice that statistically, in most of the entries of "raw_data", the right-most bit is equal to '1', since this is the measurement that follows right after applying the X gate on the qubit. The qubit thus has a negligible time to relax to the $|0 \rangle$ state until the first measurement block, and we mostly measure it in the state $|1\rangle$.

In [ ]:
len(raw_data_shots) # size of the raw_data list, equal to nr_shots

In [ ]:
len(raw_data_shots[0]) # size of the bit string of each circuit shot, equal to nr_points

In [ ]:
ro_assignment_matrix = extract_ro_assignment_matrix(ro_mitigation_shots, [qubit_nr])

raw_data_counts = get_multi_qubit_counts(raw_data_shots, len([qubit_nr]))
raw_data_probs = get_multi_qubit_prob(raw_data_counts)
ro_corrected_probs = ro_corrected_multi_qubit_prob(raw_data_probs, ro_assignment_matrix, [qubit_nr])

In [ ]:
plot_ro_assignment_matrix(ro_assignment_matrix, [qubit_nr])

Similarly to obtaining the measurement counts of a quantum circuit with a single measurement, we can do the same for all measurement block by running the custom function "get_raw_data_counts()" from utility_functions.py,

The first entry of the "raw_data_counts", raw_data_counts[0], corresponds to the total counts of the first measurement block of the quantum circuit, while the last entry raw_data_counts[-1] corresponds to the total counts of the final measurement block.

By running the following custom function, "get_raw_data_prob()", from utility_functions.py, we convert the counts of each measurement block to probabilities.

Finally, we plot the probability of the qubit being in the state $|1\rangle$ for each measurement block, which decreases exponentially with time. By fitting an exponential curve on the measurement data, we extract the relaxation time $T_1$ of the qubit,

In [ ]:
probabilities_excited = [ro_corrected_probs[entry]['1'] for entry in range(nr_points)]

def exponential_func(x, a, b):
    return a * np.exp(b*x)
params, covariance = curve_fit(exponential_func,
                               measurement_times,
                               probabilities_excited)
a_fit, b_fit = params
exponential_fit = exponential_func(measurement_times, a_fit, b_fit)

fig, ax = plt.subplots(dpi=300)
ax.scatter(1e6*measurement_times,
           probabilities_excited,
           label='Data', alpha=0.9)
ax.plot(1e6*measurement_times, exponential_fit, label='exp() fit', color='r')
ax.set_xlabel('Time (μs)')
ax.set_ylabel(r'$P(|1\rangle)$')
ax.set_title(f'T1 measurement\nQubit "Q{qubit_nr}"')

ax.text(x = 0.5*ax.get_xlim()[1], y = 0.95*ax.get_ylim()[1], s = f'T1 = {- 1e6 * 1 / b_fit:.1f} μs',
        bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 5})
ax.set_ylim(-0.05, 1.05)

plt.legend()
plt.show()

# 5. Performing a Rabi oscillation using mid-circuit measurements

In order to perform a Rabi oscillation on a qubit, we need to apply a transversal gate (we choose gate X) of varied amplitude for different measurement steps. A varied amplitude in our case translates to a varied angle of the applied X rotation.

By consulting the Starmon-7 [fact sheet](https://github.com/DiCarloLab-Delft/QuantumInspireUtilities/blob/main/Starmon7_FactSheet.pdf), we find in section VI "Qubit operations" that all x-axis and y-axis rotations, $R_{x}(\theta)$ and $R_{y}(\theta)$ respectively, can be applied with $\theta$ being any multiple of $\pi / 28$.

This means that the rotation angle cannot take any other value in between, and any angle that is not a multiple of $\pi / 28$ will be rounded to the nearest multiple of $\pi / 28$. With this experiment, we will demonstrate what happens when we try to go beyond this hardware limitation.

In a similar fashion to what we did in Section 4., we will first define the experiment parameters,

In [ ]:
qubit_nr = 0 # e.g. the entry "0" is for qubit "Q0"
angle_step = np.pi / 28
total_steps = 56

In [ ]:
qc = QuantumCircuit(7, total_steps)

for step_idx in range(total_steps):
    qc.delay(6000, qubit_nr) # initialization time for the qubit, equivalent to 120 μs
    qc.rx(step_idx * angle_step, qubit_nr)
    qc.measure(qubit = qubit_nr, cbit = step_idx)

qc = apply_readout_circuit(qc, [qubit_nr])

In [ ]:
qc.draw('mpl')

Now let us run the above circuit. Notice how the nr_shots has been set to 5500, since we are using multiple measurement blocks and we risk having the job fail if we request too many.

In [ ]:
nr_shots = 5000 # NOTE: one has to be careful with nr_shots using mid-circuit measurements
                #       if the job fails to be executed, reduce the nr_shots
job = backend.run(qc, shots=nr_shots, memory = True)
result = job.result(timeout = 600)

We obtain the probabilities for each measurement block, and visualize them

In [ ]:
raw_data_shots, ro_mitigation_shots = split_raw_shots(result, [qubit_nr])
ro_assignment_matrix = extract_ro_assignment_matrix(ro_mitigation_shots, [qubit_nr])

raw_data_counts = get_multi_qubit_counts(raw_data_shots, len([qubit_nr]))
raw_data_probs = get_multi_qubit_prob(raw_data_counts)
ro_corrected_probs = ro_corrected_multi_qubit_prob(raw_data_probs, ro_assignment_matrix, [qubit_nr])

In [ ]:
plot_ro_assignment_matrix(ro_assignment_matrix, [qubit_nr])

In [ ]:
probabilities_excited = [ro_corrected_probs[entry]['1'] for entry in range(total_steps)]

def cos_func(x, a, b, c, d):
    return a * np.cos(2*np.pi*b*x + c) + d
params, covariance = curve_fit(cos_func,
                               np.arange(0, total_steps, 1),
                               probabilities_excited)
a_fit, b_fit, c_fit, d_fit = params
cosine_fit = cos_func(np.arange(0, total_steps, 1), a_fit, b_fit, c_fit, d_fit)

fig, ax = plt.subplots(figsize=(18, 5), dpi=300)
ax.scatter(np.arange(0, total_steps, 1),
           probabilities_excited,
           label='Data', alpha=0.9)
ax.plot(np.arange(0, total_steps, 1), cosine_fit, label='cos() fit', color='r')
ax.set_xlabel('Applied rotation')
ax.set_ylabel(r'$P(|1\rangle)$')
ax.set_title(f'Rabi oscillation\nQubit "Q{qubit_nr}"')

labels = []
for step_idx in range(total_steps):
    angle_in_degrees = (360 / (2 * np.pi)) * step_idx * angle_step
    step_label = f"rx{round(angle_in_degrees)}"
    labels.append(step_label)
label_locs = np.arange(0, total_steps, 1)
ax.set_xticks(label_locs)
ax.set_xticklabels(labels, rotation=65)
ax.set_ylim(-0.05, 1.05)

plt.grid()
plt.legend()
plt.show()